## Bibliotecas

In [1]:
import pathlib, glob
from haystack.nodes import PDFToTextConverter, PreProcessor,EmbeddingRetriever
from haystack.nodes.base import BaseComponent
from haystack.document_stores import FAISSDocumentStore
from haystack.pipelines import Pipeline

/home/luid/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Pré-Processamento

In [3]:
document_store = FAISSDocumentStore(similarity="cosine")
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["pt"])
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="sentence",
    split_respect_sentence_boundary=False)
retriever = EmbeddingRetriever(
    document_store=document_store, 
    embedding_model="sentence-transformers/distiluse-base-multilingual-cased-v1")

/home/luid/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
indexing_pipeline = Pipeline()
indexing_pipeline.add_node(component=converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["TextConverter"])

NameError: name 'Pipeline' is not defined

In [ ]:
files_to_index = [f for f in pathlib.Path().glob("./ETL/paper*.pdf")]
indexing_pipeline.run(file_paths = files_to_index)

In [ ]:
document_store.update_embeddings(retriever)

In [ ]:
converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["pt"]
)

In [ ]:
from haystack.nodes import PDFToTextConverter
import langdetect

converter = PDFToTextConverter(
    remove_numeric_tables=True,
    valid_languages=["pt"]
)
docs = converter.convert(file_path="./ETL/paper1.pdf", meta=None)

In [ ]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="sentence",
    split_respect_sentence_boundary=False)

docs_default = preprocessor.process(docs)

In [ ]:
docs_default

In [ ]:
sentences = []
for doc in docs_default:
    sentences.append(doc["content"].split(". "))

In [ ]:

print(f"n_files_input: {len(docs_default)}\nn_docs_output: {len(docs_default)}")

### Verificar disponibilidade de placa gráfica

In [ ]:
torch.cuda.is_available()

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

## Modelagem

### Codificação

In [ ]:
# Codifique as sentenças usando o modelo
embeddings = model.encode(sentences, show_progress_bar=True, convert_to_tensor=True, device=device)

### Teste

In [ ]:
new_sentence = "A amazôniza azul não possui extensão de 30000 km^2"
new_embedding = model.encode([new_sentence], convert_to_tensor=True, device=device)

In [ ]:
np_embeddings = embeddings.cpu().numpy()
np_input = new_embedding.cpu().numpy()

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Calcule a similaridade de cosseno entre a nova sentença e as sentenças existentes
similarities = util.pytorch_cos_sim(new_embedding, embeddings)[0]

# Encontre os índices das sentenças mais similares (N sentenças)
N = 5
most_similar_indices = (-similarities).argsort()[:N]

In [ ]:
for idx in most_similar_indices:
    print(sentences[idx])

In [ ]:
#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")